In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

from pid import PidPathFollower
from utils import circular_trajectory, show_animation, stadium_trajectory, plot_trajectory
from vehicle import Control, State, Vehicle

In [ ]:
vehicle = Vehicle()
states = [vehicle.state.copy()]

num_steps = 100
for i in range(num_steps):
    states.append(vehicle.update(u={"steer": 0.3 * np.sin(i / 3.0) + 0.2, "accel": 2 * np.sin(i / 10.0)}))

states = pd.DataFrame(states)
plot_trajectory(states=states)

In [ ]:
def simulate(init_state: State, control_func, duration: float, dt: float, max_steer, max_accel):
    vehicle = Vehicle(init_state=init_state, max_steer=max_steer, max_accel=max_accel, dt=dt)

    states = [vehicle.state]
    controls = [{"steer": 0.0, "accel": 0.0}]

    num_steps = int(duration / dt)
    for _ in tqdm(range(num_steps)):
        try:
            control: Control = control_func(vehicle.state)
        except ValueError as e:
            print(e)
            break

        state = vehicle.update(control)

        states.append(state)
        controls.append(control)

    return pd.DataFrame(states), pd.DataFrame(controls)

In [ ]:
def normalize_angle(angle: float) -> float:
    """
    Normalize an angle to the range [-pi, pi].

    Args:
        angle (float): Angle in radians.

    Returns:
        float: Normalized angle in radians.
    """
    return (angle + np.pi) % (2 * np.pi) - np.pi


for angle in range(0, 360, 10):
    angle = np.deg2rad(angle)
    print(angle, normalize_angle(angle))

In [ ]:
duration = 250
dt = 0.1

reference = stadium_trajectory()
# reference = circular_trajectory()

init_state = dict(reference.iloc[0])
init_state["v"] = 0.0

Kp_steer = 3.0
Ki_steer = 0.1
Kd_steer = 0.3
max_steer = np.deg2rad(40)

Kp_accel = 0.4
Ki_accel = 0.0
Kd_accel = 0.0
max_accel = 2.0

pid_controller = PidPathFollower(
    reference,
    dt,
    Kp_steer,
    Ki_steer,
    Kd_steer,
    max_steer,
    Kp_accel,
    Ki_accel,
    Kd_accel,
    max_accel,
)

states, controls = simulate(init_state, pid_controller, duration, dt, max_steer, max_accel)

plot_trajectory(ref_traj=reference, states=states)

In [ ]:
skip = 32
show_animation(reference, states.iloc[::skip], controls[::skip], vehicle.max_steer, vehicle.max_accel, dt * 1000)

In [ ]:
controls[["accel", "steer"]].plot();